# ** Using Machine Learning to try and beat the bookmakers **

In [102]:
import pandas as pd
import numpy as np
from tqdm import tqdm

**#1** Collecting Data

The data comes in csv files with files being split up for each season and league.
Luckily each csv file is easily identifiable by a league code and a season code.

In [103]:
datasource_website = "http://www.football-data.co.uk/mmz4281/"


LeagueCodes = ["E0","SP1","I1","D1","F1","N1"]
Seasons = ["0405","0506","0607","0708","0809","0910","1112","1213","1314","1415","1516","1617","1718"]

EnglishLeagues = []
SpanishLeagues = []
ItalianLeagues = []
GermanLeagues = []
FrenchLeagues = []
DutchLeagues = []



Cols = ['Div','HomeTeam','AwayTeam','FTHG','FTR','FTAG','B365H','B365D','B365A','BWH','BWD','BWA']

AllLeagues = [EnglishLeagues,SpanishLeagues,ItalianLeagues,GermanLeagues,FrenchLeagues,DutchLeagues]


Downloads the CSVs and stores them as an array of dataframes

In [104]:
def group_databases(leaguearray, countrycode):
    for season in Seasons:
        url = datasource_website+"/"+ season +"/"+ countrycode
        seasondata = pd.read_csv(url,usecols = Cols,encoding = "ISO-8859-1")      
        leaguearray.append(seasondata)
    return leaguearray

In [105]:
for i in tqdm(range(0,len(AllLeagues))):
    group_databases(AllLeagues[i],LeagueCodes[i])
    


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:14<00:00,  2.41s/it]


**#2** Cleaning Data

In [106]:
#Checking for null values in any of the DataFrames 

for i in range(0, len(AllLeagues)):
    for j in range(0, len(Seasons)):
        if(AllLeagues[i][j].isnull().values.any()) == True:
            print("Null values in {} {}, Index {} {} ".format(LeagueCodes[i], Seasons[j], i, j))

Null values in E0 1415, Index 0 9 
Null values in E0 1516, Index 0 10 
Null values in I1 0405, Index 2 0 
Null values in I1 0506, Index 2 1 
Null values in I1 0809, Index 2 4 
Null values in I1 0910, Index 2 5 
Null values in I1 1213, Index 2 7 
Null values in I1 1415, Index 2 9 
Null values in I1 1516, Index 2 10 
Null values in F1 0405, Index 4 0 
Null values in F1 1314, Index 4 8 
Null values in F1 1516, Index 4 10 
Null values in N1 0405, Index 5 0 
Null values in N1 1213, Index 5 7 


It appears we have extra rows in our datasets, as we know what the true size should be for each DataFrame (360 for 20 team leagues and 306 for 18 team leagues).



In [107]:
for i in range(0, len(AllLeagues)):
    for j in range(0, len(Seasons)):
        display(len(AllLeagues[i][j]))

380

380

380

380

380

...

306

306

306

306

306

Clean excess lines in the data

In [108]:
EnglishLeagues[9] = EnglishLeagues[9].drop([380])
ItalianLeagues[7] = ItalianLeagues[7].drop([380,381,382])
ItalianLeagues[9] = ItalianLeagues[9].drop([380])
ItalianLeagues[10] = ItalianLeagues[10].drop([380])
FrenchLeagues[8] = FrenchLeagues[8].drop([380])
FrenchLeagues[10] = FrenchLeagues[10].drop([380])
DutchLeagues[7] = DutchLeagues[7].drop([306,307,308])


In [109]:
for i in range(0, len(AllLeagues)):
    for j in range(0, len(Seasons)):
        if(AllLeagues[i][j].isnull().values.any()) == True:
            print("Null values in {} {}, Index {} {} ".format(LeagueCodes[i], Seasons[j], i, j))

Null values in E0 1516, Index 0 10 
Null values in I1 0405, Index 2 0 
Null values in I1 0506, Index 2 1 
Null values in I1 0809, Index 2 4 
Null values in I1 0910, Index 2 5 
Null values in I1 1213, Index 2 7 
Null values in F1 0405, Index 4 0 
Null values in F1 1516, Index 4 10 
Null values in N1 0405, Index 5 0 


In [110]:
#All other null values are from missing b365 DATA

In [111]:
def resetindex(data):
    for sets in data:
        sets.index = range(0,len(sets))
    
resetindex(EnglishLeagues)
resetindex(SpanishLeagues)
resetindex(ItalianLeagues)
resetindex(GermanLeagues)
resetindex(FrenchLeagues)
resetindex(DutchLeagues)

**#3** FeatureCreation

In [112]:
def GetMatchdays(data):
    return int(len(data) * 2 / len(data.groupby('HomeTeam').mean().T.columns))

def FilterFirst5(data):
    data = data[(data.HomeTeamMatchDay > 8) & (data.AwayTeamMatchDay > 8)]
    
    return data

def Aggregate(data, matchdays):
    
    df = pd.DataFrame.from_dict(data,orient = 'index',columns = [i for i in range(1,matchdays + 1)])
    df.transpose()
    df_last5 = pd.DataFrame()
    
    df_last5[1] = (df[1])
    df_last5[2] = (df[1]) * 5
    df_last5[3] = (df[2] + df[1]) * 5/2
    df_last5[4] = (df[3] + df[2] + df[1]) * 5/3
    df_last5[5] = (df[4] + df[3] + df[2] + df[1]) * 5/4
    
    for i in range(6,matchdays+1):
        df_last5[i] = df[i-1] + df[i-2] + df[i-3] + df[i - 4] + df[i - 5]
    
    return df_last5

In [113]:
##Basic PreProcessing Cell

def Get_GoalsScoredLast5(data):
    
    matchdays = GetMatchdays(data)  
     
    teams = {}
    
    for i in data.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
        
   
    for i in range(len(data)):
        HTGS = data.iloc[i]['FTHG']
        ATGS = data.iloc[i]['FTAG']
        teams[data.iloc[i].HomeTeam].append(HTGS)
        teams[data.iloc[i].AwayTeam].append(ATGS)
        
    
    GoalsScoredLast5 = Aggregate(teams, matchdays)
    
    return GoalsScoredLast5

def Get_UnBeatenStreak(data):
    
    matchdays = GetMatchdays(data)   
    teams = {}
    
    for i in data.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
        
   
    for i in range(len(data)):
        FTR = data.iloc[i]['FTR']
        if FTR == 'H':
            teams[data.iloc[i].HomeTeam].append(1)
            teams[data.iloc[i].AwayTeam].append(0)
        if FTR == 'D':
            teams[data.iloc[i].HomeTeam].append(1)
            teams[data.iloc[i].AwayTeam].append(1)
        if FTR == 'A':
            teams[data.iloc[i].HomeTeam].append(0)
            teams[data.iloc[i].AwayTeam].append(1)
        
    
    NotLose = pd.DataFrame.from_dict(teams,orient = 'index',columns = [i for i in range(1,matchdays + 1)])
    NotLose.transpose()
    UnBeatenStreak = pd.DataFrame.from_dict(teams,orient = 'index',columns = [i for i in range(1,matchdays + 1)])
    UnBeatenStreak.transpose()
     
    UnBeatenStreak[1] = 0
    for i in range(2,(matchdays + 1)):
        for column in teams:
            if NotLose[i - 1][column] == 1:
                UnBeatenStreak[i][column] = UnBeatenStreak[i - 1][column] + 1
            else:
                UnBeatenStreak[i][column] = 0
    return UnBeatenStreak




def Get_GoalsConcededLast5(data):
    
    matchdays = GetMatchdays(data)
    teams = {}
                           
    for i in data.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
        
   
    for i in range(len(data)):
        HTGS = data.iloc[i]['FTHG']
        ATGS = data.iloc[i]['FTAG']
        teams[data.iloc[i].HomeTeam].append(ATGS)
        teams[data.iloc[i].AwayTeam].append(HTGS)
        
    
    GoalsConcededLast5 = Aggregate(teams, matchdays)
        
    return GoalsConcededLast5
    
def Get_Points(data):
    matchdays = GetMatchdays(data)
    teams = {}
    
    for i in data.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
        
    for i in range(len(data)):
        FTR = data.iloc[i]['FTR']
        if FTR == 'H':
            teams[data.iloc[i].HomeTeam].append(3)
            teams[data.iloc[i].AwayTeam].append(0)
        if FTR == 'D':
            teams[data.iloc[i].HomeTeam].append(1)
            teams[data.iloc[i].AwayTeam].append(1)
        if FTR == 'A':
            teams[data.iloc[i].HomeTeam].append(0)
            teams[data.iloc[i].AwayTeam].append(3)
            
    PointsLast5 = Aggregate(teams, matchdays)
        
    return PointsLast5

def LostLast(data):
    matchdays = GetMatchdays(data)
    teams = {}
    
    for i in data.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
        
    for i in range(len(data)):
        FTR = data.iloc[i]['FTR']
        if FTR == 'H':
            teams[data.iloc[i].HomeTeam].append(1)
            teams[data.iloc[i].AwayTeam].append(0)
        if FTR == 'D':
            teams[data.iloc[i].HomeTeam].append(1)
            teams[data.iloc[i].AwayTeam].append(1)
        if FTR == 'A':
            teams[data.iloc[i].HomeTeam].append(0)
            teams[data.iloc[i].AwayTeam].append(1)
     
    df = pd.DataFrame.from_dict(teams,orient = 'index',columns = [i for i in range(1,matchdays + 1)])
    df.transpose()
    
    LostLast = pd.DataFrame()
    
    LostLast[1] = 0
    
    for i in range(2,matchdays + 1):
        LostLast[i] = df[i - 1]
        
    return LostLast
    

def Get_HomeTeamPoints(data):
    matchdays = int(GetMatchdays(data)/2)
    hometeams = {}
    awayteams = {}
    
    for i in data.groupby('HomeTeam').mean().T.columns:
        hometeams[i] = []
        awayteams[i] = []
        
    for i in range(len(data)):
        FTR = data.iloc[i]['FTR']
        if FTR == 'H':
            hometeams[data.iloc[i].HomeTeam].append(3)
            awayteams[data.iloc[i].AwayTeam].append(0)
        if FTR == 'D':
            hometeams[data.iloc[i].HomeTeam].append(1)
            awayteams[data.iloc[i].AwayTeam].append(1)
        if FTR == 'A':
            hometeams[data.iloc[i].HomeTeam].append(0)
            awayteams[data.iloc[i].AwayTeam].append(3)
            
    HomeTeamPoints5 = Aggregate(hometeams, matchdays)
    AwayTeamPoints5 = Aggregate(awayteams, matchdays)
        
    return HomeTeamPoints5, AwayTeamPoints5

def Get_HomeTeamGoalsScoredLast5(data):
    
    matchdays = int(GetMatchdays(data)/2)
    hometeams = {}
    awayteams = {}
    
    for i in data.groupby('HomeTeam').mean().T.columns:
        hometeams[i] = []
        awayteams[i] = []
        
   
    for i in range(len(data)):
        HTGS = data.iloc[i]['FTHG']
        ATGS = data.iloc[i]['FTAG']
        hometeams[data.iloc[i].HomeTeam].append(HTGS)
        awayteams[data.iloc[i].AwayTeam].append(ATGS)
        
    
    HomeTeamGoalsScored5 = Aggregate(hometeams, matchdays)
    AwayTeamGoalsScored5 = Aggregate(awayteams, matchdays)
            
        
    return HomeTeamGoalsScored5, AwayTeamGoalsScored5
  


In [114]:
def ApplyingAllStats(data):
    homematch =[]
    awaymatch =[]
    j = data.index.values[0]
    #Find the current matchday for the home team and away team, to aid with filtering down the line
    for i in range(len(data)):
        homematch.append(len(data[(data.index.values <= i) & ((data['HomeTeam'] == data.iloc[i].HomeTeam) | (data['AwayTeam'] == data.iloc[i].HomeTeam))]))

    for i in range(len(data)):
        awaymatch.append(len(data[(data.index.values <= i) & ((data['HomeTeam'] == data.iloc[i].AwayTeam) | (data['AwayTeam'] == data.iloc[i].AwayTeam))]))

    data['HomeTeamMatchDay'] = homematch
    data['AwayTeamMatchDay'] = awaymatch
    
    V = LostLast(data)
    W = Get_Points(data)
    Y = Get_GoalsScoredLast5(data)
    X = Get_GoalsConcededLast5(data)
    Z = Get_UnBeatenStreak(data)
    homePoints = []
    AwayTeamPoints = []
    homeGoalsScoredLast5 = []
    AwayTeamGoalsScoredLast5 = []
    homeGoalsConcededLast5 = []
    AwayTeamGoalsConcededLast5 = []
    homeUnBeatenStreak = []
    AwayTeamUnBeatenStreak = []
    homeOverUnder = []
    AwayTeamOverUnder = []
    homelostlast = []
    AwayTeamlostlast = []
    
    for i in range(len(data)):
        A = data['HomeTeam'][i + j]
        B = data['HomeTeamMatchDay'][i + j] 
        homeGoalsScoredLast5.append(Y.loc[A,B])
        homeGoalsConcededLast5.append(X.loc[A,B])
        homeUnBeatenStreak.append(Z.loc[A,B])
        homePoints.append(W.loc[A,B])
        homelostlast.append(V.loc[A,B])
        
        
        A = data['AwayTeam'][i + j]
        B = data['AwayTeamMatchDay'][i + j]
        AwayTeamGoalsScoredLast5.append(Y.loc[A,B])
        AwayTeamGoalsConcededLast5.append(X.loc[A,B])
        AwayTeamUnBeatenStreak.append(Z.loc[A,B])
        AwayTeamPoints.append(W.loc[A,B])
        AwayTeamlostlast.append(V.loc[A,B])
  
    
    #Convert all the generated arrays into columns for the main dataframe
    data['HomeTeamGoalsScoredLast5'] = homeGoalsScoredLast5
    data['AwayTeamGoalsScoredLast5'] = AwayTeamGoalsScoredLast5
    data['HomeTeamGoalsConcededLast5'] = homeGoalsConcededLast5
    data['AwayTeamGoalsConcededLast5'] = AwayTeamGoalsConcededLast5
    data['HomeTeamUnBeatenStreak'] = homeUnBeatenStreak
    data['AwayTeamUnBeatenStreak'] = AwayTeamUnBeatenStreak
    data['HomeTeamPoints'] = homePoints
    data['AwayTeamPoints'] = AwayTeamPoints
    data['HomeTeamLostLast'] = homelostlast
    data['AwayTeamLostLast'] = AwayTeamlostlast
    
    
    #Using the matchday column to filter out the first x amount of games, these early matchdays can be problematic due to weak -
    #Definitions of the form metrics
    FilterFirst5(data)
    
    return data

def ApplyingHomeTeamStats(data):
    homematch =[]
    awaymatch =[]
    j = data.index.values[0]
    for i in range(len(data)):
        homematch.append(len(data[(data.index.values <= i) & ((data['HomeTeam'] == data.iloc[i].HomeTeam))]))

    for i in range(len(data)):
        awaymatch.append(len(data[(data.index.values <= i) & ((data['AwayTeam'] == data.iloc[i].AwayTeam))]))

    data['HomeTeamGameNumber'] = homematch
    data['AwayTeamGameNumber'] = awaymatch
    
    
    X,W = Get_HomeTeamPoints(data)
    Y,Z = Get_HomeTeamGoalsScoredLast5(data)

  
    homePoints = []
    AwayTeamPoints = []
    homeGoalsScoredLast5 = []
    AwayTeamGoalsScoredLast5 = []
    
    
    for i in range(len(data)):
        A = data['HomeTeam'][i + j]
        B = data['HomeTeamGameNumber'][i + j] 
        homeGoalsScoredLast5.append(Y.loc[A,B])
        homePoints.append(X.loc[A,B])
        
        A = data['AwayTeam'][i + j]
        B = data['AwayTeamGameNumber'][i + j]
        AwayTeamGoalsScoredLast5.append(Z.loc[A,B])
        AwayTeamPoints.append(X.loc[A,B])
    
    
    data['ScoredAtHomeTeam5'] = homeGoalsScoredLast5
    data['ScoredAwayTeam5'] = AwayTeamGoalsScoredLast5
    data['HomeTeamAtHomeTeamPoints'] = homePoints
    data['AwayTeamFromHomeTeamPoints'] = AwayTeamPoints

    return data


In [115]:
def ApplyStatsToArray(array):
    for i in range(0,len(array)):
        array[i] = ApplyingAllStats(array[i])
        array[i] = ApplyingHomeTeamStats(array[i])

    array.pop(0)
    return array


In [116]:
EnglishLeagues = ApplyStatsToArray(EnglishLeagues)


In [117]:

GermanLeagues = ApplyStatsToArray(GermanLeagues)
FrenchLeagues = ApplyStatsToArray(FrenchLeagues)
DutchLeagues = ApplyStatsToArray(DutchLeagues)


In [118]:

SpanishLeagues = ApplyStatsToArray(SpanishLeagues)
ItalianLeagues = ApplyStatsToArray(ItalianLeagues)

In [119]:
EnglandFinal = pd.concat(EnglishLeagues)
SpainFinal = pd.concat(SpanishLeagues)
GermanyFinal = pd.concat(ItalianLeagues)
ItalyFinal = pd.concat(GermanLeagues)
FranceFinal = pd.concat(FrenchLeagues)
NetherlandsFinal = pd.concat(DutchLeagues)

In [120]:
Final = pd.concat([EnglandFinal,SpainFinal,GermanyFinal,ItalyFinal,FranceFinal,NetherlandsFinal])

In [121]:
Final.to_csv()

",Div,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,BWH,BWD,BWA,HomeTeamMatchDay,AwayTeamMatchDay,HomeTeamGoalsScoredLast5,AwayTeamGoalsScoredLast5,HomeTeamGoalsConcededLast5,AwayTeamGoalsConcededLast5,HomeTeamUnBeatenStreak,AwayTeamUnBeatenStreak,HomeTeamPoints,AwayTeamPoints,HomeTeamLostLast,AwayTeamLostLast,HomeTeamGameNumber,AwayTeamGameNumber,ScoredAtHomeTeam5,ScoredAwayTeam5,HomeTeamAtHomeTeamPoints,AwayTeamFromHomeTeamPoints\r\n0,E0,Aston Villa,Bolton,2.0,2.0,D,2.3,3.25,3.0,2.1,3.25,3.15,1,1,2.0,2.0,2.0,2.0,0,0,1.0,1.0,,,1,1,2.0,2.0,1.0,0.0\r\n1,E0,Everton,Man United,0.0,2.0,A,5.0,3.4,1.72,4.35,3.35,1.75,1,1,0.0,2.0,2.0,0.0,0,0,0.0,3.0,,,1,1,0.0,2.0,0.0,3.0\r\n2,E0,Fulham,Birmingham,0.0,0.0,D,2.37,3.25,2.87,2.3,3.2,2.8,1,1,0.0,0.0,0.0,0.0,0,0,1.0,1.0,,,1,1,0.0,0.0,1.0,0.0\r\n3,E0,Man City,West Brom,0.0,0.0,D,1.72,3.4,5.0,1.65,3.45,4.8,1,1,0.0,0.0,0.0,0.0,0,0,1.0,1.0,,,1,1,0.0,0.0,1.0,3.0\r\n4,E0,Middlesbrough,Liverpool,0.0,0.0,D,2.87,3.2,2.4,2.9,3.35,2.2,1,1,0.0,0.0,0.0,0.0,

In [122]:
display(Final)

ImportantCols = ['FTHG','FTAG','HomeTeamMatchDay','AwayTeamMatchDay','HomeTeamPoints','AwayTeamPoints','Hom']

Final = Final.sample(frac=1).reset_index(drop=True)

,Div,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,BWH,...,HomeTeamPoints,AwayTeamPoints,HomeTeamLostLast,AwayTeamLostLast,HomeTeamGameNumber,AwayTeamGameNumber,ScoredAtHomeTeam5,ScoredAwayTeam5,HomeTeamAtHomeTeamPoints,AwayTeamFromHomeTeamPoints
0,E0,Aston Villa,Bolton,2.0,2.0,D,2.30,3.25,3.00,2.10,...,1.0,1.0,NaN,NaN,1,1,2.0,2.0,1.0,0.0
1,E0,Everton,Man United,0.0,2.0,A,5.00,3.40,1.72,4.35,...,0.0,3.0,NaN,NaN,1,1,0.0,2.0,0.0,3.0
2,E0,Fulham,Birmingham,0.0,0.0,D,2.37,3.25,2.87,2.30,...,1.0,1.0,NaN,NaN,1,1,0.0,0.0,1.0,0.0
3,E0,Man City,West Brom,0.0,0.0,D,1.72,3.40,5.00,1.65,...,1.0,1.0,NaN,NaN,1,1,0.0,0.0,1.0,3.0
4,E0,Middlesbrough,Liverpool,0.0,0.0,D,2.87,3.20,2.40,2.90,...,1.0,1.0,NaN,NaN,1,1,0.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,N1,Roda,Den Haag,2.0,3.0,A,2.89,3.60,2.25,2.95,...,10.0,5.0,1.0,1.0,17,17,6.0,6.0,4.0,9.0
302,N1,Sparta Rotterdam,Heracles,2.0,5.0,A,2.14,3.79,3.00,2.10,...,6.0,2.0,0.0,1.0,17,17,10.0,5.0,9.0,9.0
303,N1,Twente,NAC Breda,1.0,1.0,D,2.14,3.75,3.10,2.15,...,4.0,6.0,0.0,1.0,17,17,7.0,5.0,6.0,6.0
304,N1,Utrecht,VVV Venlo,1.0,0.0,H,1.44,4.50,6.50,1.48,...,3.0,1.0,1.0,0.0,17,17,14.0,4.0,11.0,3.0


In [123]:
def FilterFirst5(data):
    data = data[(data.HomeTeamMatchDay > 8) & (data.AwayTeamMatchDay > 8)]
    
    return data

def FilterFirst6(data):
    data = data[(data.B365A > 4.4) & (data.B365A < 5.6)]
    
    return data

In [124]:
Final5 = FilterFirst5(Final)
Final5 = FilterFirst6(Final5)
Final5.index = range(0,len(Final5))

In [125]:
def SplitOutStats(data):
  
    H = []
    D = []
    A = []
    
    for i in range(len(data)):
        if data['FTR'][i] == 'H':
            H.append(1)
            D.append(0)
            A.append(0)
        elif data['FTR'][i] == 'D':
            H.append(0)
            D.append(1)
            A.append(0)
        else:
            H.append(0)
            D.append(0)
            A.append(1)
    data['H'] = H
    data['D'] = D
    data['A'] = A
    
    
    
    
    ArrangeCols = ['B365H','B365D','B365A','H','D','A','HomeTeamAtHomeTeamPoints','AwayTeamFromHomeTeamPoints','ScoredAtHomeTeam5',
                   'ScoredAwayTeam5','HomeTeamPoints','AwayTeamPoints','HomeTeamUnBeatenStreak','AwayTeamUnBeatenStreak',
                   'HomeTeamGoalsScoredLast5','AwayTeamGoalsScoredLast5','HomeTeamGoalsConcededLast5',
                   'AwayTeamGoalsConcededLast5','HomeTeamLostLast','AwayTeamLostLast']
    data = data[ArrangeCols]    
    return data

def SplitOutOdds(data):
    
    Odds = pd.DataFrame()
    
    Odds['OddsH'] = data['B365H']
    Odds['OddsD'] = data['B365D']
    Odds['OddsA'] = data['B365A']
    
    return Odds
    

In [126]:
Final55 = SplitOutStats(Final5)
Odds55 = SplitOutOdds(Final5)


In [127]:
display(Final55)

,B365H,B365D,B365A,H,D,A,HomeTeamAtHomeTeamPoints,AwayTeamFromHomeTeamPoints,ScoredAtHomeTeam5,ScoredAwayTeam5,HomeTeamPoints,AwayTeamPoints,HomeTeamUnBeatenStreak,AwayTeamUnBeatenStreak,HomeTeamGoalsScoredLast5,AwayTeamGoalsScoredLast5,HomeTeamGoalsConcededLast5,AwayTeamGoalsConcededLast5,HomeTeamLostLast,AwayTeamLostLast
0,1.73,3.40,5.25,1,0,0,8.00,8.00,17.0,6.00,8.0,13.0,2,6,11.0,6.0,5.0,1.0,1.0,1.0
1,1.90,3.00,4.50,1,0,0,6.00,4.00,5.0,3.00,7.0,0.0,0,0,8.0,1.0,9.0,7.0,0.0,0.0
2,1.73,3.40,5.25,1,0,0,13.00,11.00,11.0,6.00,11.0,10.0,5,1,7.0,10.0,2.0,4.0,1.0,1.0
3,1.72,3.00,5.00,1,0,0,10.00,7.00,7.0,2.00,3.0,7.0,0,6,1.0,4.0,7.0,3.0,0.0,1.0
4,1.66,3.60,5.25,0,1,0,10.00,11.00,10.0,7.00,10.0,10.0,1,0,10.0,14.0,4.0,8.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2375,1.80,3.60,5.25,0,0,1,5.00,6.00,3.0,6.00,5.0,3.0,0,1,3.0,5.0,4.0,10.0,0.0,1.0
2376,1.66,3.40,5.50,0,0,1,7.00,4.00,7.0,4.00,6.0,5.0,0,1,5.0,9.0,6.0,9.0,0.0,1.0
2377,1.72,3.60,5.00,0,1,0,10.00,10.00,6.0,2.00,12.0,6.0,0,1,7.0,3.0,2.0,13.0,0.0,1.0
2378,1.57,4.33,5.50,1,0,0,8.75,0.00,5.0,10.00,8.0,7.0,4,2,5.0,7.0,4.0,7.0,1.0,1.0


In [128]:
Final55.to_csv('Data/DataBase.csv')
Odds55.to_csv('Data/Odds.csv')
Final.to_csv('Data/Info.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Data/DataBase.csv'

In [ ]:
def FilterFirst5(data):
    data = data[(data.HomeTeamMatchDay > 8) & (data.AwayTeamMatchDay > 8)]
    
    return data